In [1]:
from utils import *
import torch
import streamlit as st
from utils import *
from langchain.prompts import ChatPromptTemplate
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import logging
from importlib import reload


In [2]:
device = torch.device("cuda")
torch.cuda.is_available()
# torch.Tensor(1).to(device)

# reload(src/utils)

True

In [3]:
load_dotenv()  # take environment variables from .env.
index_name = "pubmedbert-sentence-transformer-400"
HUGGINGFACE_TOKEN = os.getenv('HUGGINGFACE_TOKEN')
HUGGINGFACE_USERNAME = os.getenv('HUGGINGFACE_USERNAME')
HUGGINGFACE_DATASET_NAME = os.getenv('HUGGINGFACE_DATASET_NAME')
ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')
QA_VALIDATION_DATASET = os.getenv('QA_VALIDATION_DATASET')
QA_VALIDATION_TOKEN = os.getenv('QA_VALIDATION_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
def _parse(text):
    return text.strip("**")


In [4]:

model_name = "NeuML/pubmedbert-base-embeddings"
device = 'cuda:0'
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs={'device': device}
)

elastic_vector_search = ElasticsearchStore(
        es_cloud_id = ELASTIC_CLOUD_ID,
        index_name = index_name,
        embedding = embeddings,
        es_api_key = ELASTIC_API_KEY
)

retriever = elastic_vector_search.as_retriever(search_kwargs={"k":50})


### Multi-query retriever

In [5]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()


In [6]:
template = """
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives. The user questions
are focused on Large Language Models, Machine Learning, and related
disciplines.
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template,
)
llm = Ollama(model="llama2")

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [7]:
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    verbose=True,
    retriever = elastic_vector_search.as_retriever(search_kwargs={"k":3}),
    chain_type_kwargs={
        "verbose": True },
)

In [8]:
rag("AI in medicine")

c:\Users\priot\anaconda3\envs\nlp\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

" the practice of medicine is changing with the development of new artificial intelligence ( ai ) methods of machine learning. coupled with rapid improvements in computer processing, these ai - based systems are already improving the accuracy and efficiency of diagnosis and treatment across various specializations. the increasing focus of ai in radiology has led to some experts suggesting that someday ai may even replace radiologists. these suggestions raise the question of whether ai - based systems will eventually replace physicians in some specializations or will augment the role of physicians without actually replacing them. to assess the impact on physicians this research seeks 

{'query': 'AI in medicine',
 'result': "AI (Artificial Intelligence) is increasingly being used in medicine, from imaging interpretation and health monitoring to drug development. While AI can help with diagnosis, it's important to remember that AI systems are not yet ready to replace humans when it comes to the diagnosis of sports medicine conditions. Experts still outperform AI systems for highly specialized fields like sports medicine.\n\nHere are some potential challenges and opportunities related to AI in medicine:\n\nChallenges:\n\n1. Interpretability: AI algorithms can be difficult to interpret, making it challenging to understand why a particular diagnosis or treatment recommendation was made.\n2. Data quality: AI algorithms require high-quality data to produce accurate results. However, medical data can be noisy, incomplete, or biased, which can affect the accuracy of AI models.\n3. Regulatory frameworks: Existing regulatory frameworks may not be adequate for governing the use

In [24]:
question = "AI in medicine"

In [9]:
from langchain_community.document_transformers.embeddings_redundant_filter import *

In [18]:
retriever_15 = elastic_vector_search.as_retriever(search_kwargs={"k":15})
docs = retriever_15.get_relevant_documents(question)

In [19]:
docs

[Document(page_content='" the practice of medicine is changing with the development of new artificial intelligence ( ai ) methods of machine learning. coupled with rapid improvements in computer processing, these ai - based systems are already improving the accuracy and efficiency of diagnosis and treatment across various specializations. the increasing focus of ai in radiology has led to some experts suggesting that someday ai may even replace radiologists. these suggestions raise the question of whether ai - based systems will eventually replace physicians in some specializations or will augment the role of physicians without actually replacing them. to assess the impact on physicians this research seeks to better understand this technology and how it is transforming medicine. to that end this paper researches the role of ai - based systems in performing medical work in specializations including radiology, pathology, ophthalmology, and cardiology. it concludes that ai - based systems

In [29]:
a = EmbeddingsRedundantFilter(embeddings=embeddings, threshold=0.9) ## default is cosine similarity
unique_docs = a.transform_documents(docs)

In [30]:
unique_docs = [x.to_document() for x in unique_docs] ## this have their embeddings here
unique_docs

[Document(page_content='" the practice of medicine is changing with the development of new artificial intelligence ( ai ) methods of machine learning. coupled with rapid improvements in computer processing, these ai - based systems are already improving the accuracy and efficiency of diagnosis and treatment across various specializations. the increasing focus of ai in radiology has led to some experts suggesting that someday ai may even replace radiologists. these suggestions raise the question of whether ai - based systems will eventually replace physicians in some specializations or will augment the role of physicians without actually replacing them. to assess the impact on physicians this research seeks to better understand this technology and how it is transforming medicine. to that end this paper researches the role of ai - based systems in performing medical work in specializations including radiology, pathology, ophthalmology, and cardiology. it concludes that ai - based systems

In [31]:
QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know". Limit your answers to maximum 100 words.

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

out = qa_chain(
    inputs={
        "query": question,
        "contexts": "\n---\n".join([d.page_content for d in unique_docs[:2]])
    }
)
out["text"]

"Hello! I'm here to help answer your questions about AI in medicine. Based on the contexts provided, it seems that AI is transforming the field of medicine and has the potential to augment physicians without replacing them entirely. According to the research paper, AI-based systems are already improving the accuracy and efficiency of diagnosis and treatment across various specializations, including radiology, pathology, ophthalmology, and cardiology. However, it also highlights the importance of addressing legal and ethical considerations related to data usage to ensure widespread acceptance of AI in medicine.\n\nIn terms of your question, AI is definitely changing the practice of medicine, particularly in radiology and other specialties. While some experts suggest that AI may eventually replace radiologists, it's important to recognize that AI will augment physicians rather than completely replacing them. The traditional physician-patient relationship is unlikely to be replaced by AI,

In [ ]:
## replacement to RetrievalQA.from_chain_type



In [23]:
# from langchain.retrievers.multi_query import MultiQueryRetriever

# question = "What is the relationship between risk of schizophrenia and premorbid IQ?"

# # Run
# multi_retriever = MultiQueryRetriever(
#     retriever=retriever, llm_chain=llm_chain, parser_key="lines"
# )  # "lines" is the key (attribute name) of the parsed output

# # Results
# docs = multi_retriever.get_relevant_documents(
#     query=question
# )
# len(docs)

In [4]:
rag_pipeline = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY),
        chain_type="stuff",
        verbose=True,
        retriever = retriever,
        chain_type_kwargs={
            "verbose": True },
    )

In [5]:
print(retriever.get_relevant_documents("what did he say about ketanji brown jackson"))

[Document(page_content='" until now, i have not responded to the undocumented and untrue portrayal of me in the hoffman report, or the newspaper and email accounts of my alleged role in \\ " torture. \\ " i have waited several years because i believed that the american psychological association leadership would completely disavow the hoffman report and find a way to restore my reputation as well as that of the other former american psychological association presidents and the small number of its senior staff who were also falsely accused. although this has not yet happened, i remain optimistic that it will. here i present the facts as they relate to me. "', metadata={'title': 'The American Psychological Association\'s Hoffman report allegations of my association with the "CIA torture" program.', 'publication_date': '03/09/2018', 'id': 11112, 'authors': ['Joseph D Matarazzo']}), Document(page_content='##y and know yourself, you need not fear the result of a hundred battles.\'"', metadat

In [6]:
query = "What were the most commonly used methods for bone age assessment in the studies reviewed?"

In [9]:
# rag = rag_pipeline(model_id, index_name)

## Rewrite the input text
rewrite_prompt = hub.pull("langchain-ai/rewrite")
rewrite_llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

rewriter = rewrite_prompt | rewrite_llm | StrOutputParser() | _parse
rewritten_input_text = rewriter.invoke({"x": query})

logging.info(f"Original input: {query}. Transformed input: {rewritten_input_text}")

print(rewritten_input_text)

Most commonly used methods for bone age assessment in medical studies**.


In [10]:
rag_pipeline(rewritten_input_text)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
" the accuracy and consistency of bone age assessments ( baa ) using standard methods can vary with physicians'level of experience. "

" bone age is one of biological indicators of maturity used in clinical practice and it is a very important parameter of a child \ u2019s assessment, especially in paediatric endocrinology. the most widely used method of bone age assessment is by performing a hand and wrist radiograph and its analysis with greulich - pyle or tanner - whitehouse atlases, although it has been about 60 years since they were published. due to the progress in the area of computer - aided diagnosis and application of artificial intelligence in medicine,

{'query': 'Most commonly used methods for bone age assessment in medical studies**.',
 'result': 'The most commonly used methods for bone age assessment in medical studies include performing a hand and wrist radiograph and analyzing it with Greulich-Pyle or Tanner-Whitehouse atlases. Additionally, there are now numerous programs for automatic bone age assessment that have been developed using computer-aided diagnosis and artificial intelligence in medicine. These programs have shown good precision, eliminated variability between raters, and reduced the time needed for assessment. Other methods that avoid X-ray exposure, such as ultrasound or magnetic resonance imaging, are also available for bone age assessment.'}

In [26]:
docs = retriever.get_relevant_documents(rewritten_input_text)

for doc in docs:
    print(doc.page_content)

" the accuracy and consistency of bone age assessments ( baa ) using standard methods can vary with physicians'level of experience. "
" bone age is one of biological indicators of maturity used in clinical practice and it is a very important parameter of a child \ u2019s assessment, especially in paediatric endocrinology. the most widely used method of bone age assessment is by performing a hand and wrist radiograph and its analysis with greulich - pyle or tanner - whitehouse atlases, although it has been about 60 years since they were published. due to the progress in the area of computer - aided diagnosis and application of artificial intelligence in medicine, lately, numerous programs for automatic bone age assessment have been created. most of them have been verified in clinical studies in comparison to traditional methods, showing good precision while eliminating inter - and intra - rater variability and significantly reducing the time of assessment. additionally, there are availa

In [21]:
model = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

ANSWER_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. Keep your answers shorter than 25 words.
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

ANSWER_PROMPT_medical= ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Assume that the reader has in depth knowledge into the medical field and is familiar with all the terms used, so they do not need to be explained. Keep your answers shorter than 25 words.
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT_medical
    | model
    | StrOutputParser()
)

ans = chain.invoke(rewritten_input_text)

print("---- Answer ----")
print(ans)

---- Answer ----
Hand and wrist radiograph analysis with Greulich-Pyle or Tanner-Whitehouse atlases is the most commonly used method for bone age assessment in medical studies.


In [ ]:
## The grand scientific challenge in materials research is to accelerate materials discovery and development for emerging and future technologies by integrating automation with artificial intelligence
# "Between 2019 Q4 and 2020 Q3 phenethyl 4 ANPP was detected in 25 1 054 fentanyl cases in the USA"

In [16]:
import json

with open("../data/evaluation_dataset.json", 'r') as file:
    json_data = file.read()

manual_inspection = json.loads(json_data)

for item in manual_inspection:
    ans = chain.invoke(rewriter.invoke({"x": item["question"]}))
    # print("Question: {}\nGround truth: {}\nGenerated answer: {}\n\n".format(item["question"], item["paragraph"], ans))
    item["generated_answer"] = ans


TypeError: dump() missing 1 required positional argument: 'fp'

In [18]:

with open("../data/evaluation_dataset_openai.json", 'w') as file:
    json.dump(manual_inspection,file, indent=2)